# 高频交易与订单簿分析

**市场微观结构与量化交易实战**

---

## 目录

### 第一部分：订单簿基础
1. 订单簿结构与原理
2. 买卖盘深度与流动性
3. 订单类型
4. 数据获取方式

### 第二部分：订单簿分析
5. 买卖价差分析
6. 深度不平衡指标
7. 订单流分析
8. VWAP/TWAP 计算

### 第三部分：微观结构指标
9. 流动性指标
10. 市场冲击模型
11. 价格发现机制

### 第四部分：高频策略
12. 做市策略
13. 统计套利
14. 动量/均值回归

### 第五部分：实战系统
15. WebSocket 实时数据
16. 订单簿可视化
17. 执行算法

---

# 第一部分：订单簿基础

---

## 1. 订单簿结构与原理

### 1.1 什么是订单簿？

**订单簿（Order Book）** 是交易所维护的买卖订单列表，记录了所有未成交的限价订单。

### 1.2 订单簿结构

```
┌─────────────────────────────────────────────────────────────────┐
│                        订单簿结构示意图                         │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   卖单（Asks）- 按价格从低到高排列                             │
│   ┌─────────────────────────────────────────────────┐          │
│   │  价格       数量       累计                      │          │
│   │  50,100     1.5       1.5       ← 卖一（最优卖价）│          │
│   │  50,150     2.0       3.5                        │          │
│   │  50,200     3.0       6.5                        │          │
│   │  50,250     1.0       7.5                        │          │
│   └─────────────────────────────────────────────────┘          │
│                                                                 │
│   ─────────────── 买卖价差（Spread）───────────────            │
│                                                                 │
│   买单（Bids）- 按价格从高到低排列                             │
│   ┌─────────────────────────────────────────────────┐          │
│   │  价格       数量       累计                      │          │
│   │  50,000     2.0       2.0       ← 买一（最优买价）│          │
│   │  49,950     1.5       3.5                        │          │
│   │  49,900     4.0       7.5                        │          │
│   │  49,850     2.5       10.0                       │          │
│   └─────────────────────────────────────────────────┘          │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### 1.3 关键术语

| 术语 | 定义 |
|------|------|
| **Bid（买价）** | 买方愿意支付的最高价格 |
| **Ask（卖价）** | 卖方愿意接受的最低价格 |
| **Spread（价差）** | Ask - Bid |
| **Mid Price（中间价）** | (Ask + Bid) / 2 |
| **Depth（深度）** | 各价位的订单数量 |

## 2. 买卖盘深度与流动性

### 2.1 流动性的定义

**流动性** 是指资产能够快速买卖而不显著影响价格的能力。

### 2.2 流动性的衡量

```
┌─────────────────────────────────────────────────────────────────┐
│                      流动性衡量维度                             │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  1. 宽度（Tightness）                                          │
│     └── 买卖价差越小，流动性越好                               │
│                                                                 │
│  2. 深度（Depth）                                              │
│     └── 各价位订单量越大，流动性越好                           │
│                                                                 │
│  3. 弹性（Resiliency）                                         │
│     └── 大单成交后价格恢复速度                                 │
│                                                                 │
│  4. 即时性（Immediacy）                                        │
│     └── 订单执行的速度                                         │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

## 3. 订单类型

| 订单类型 | 描述 | 使用场景 |
|----------|------|----------|
| **市价单** | 立即以最优价格成交 | 需要立即执行 |
| **限价单** | 指定价格挂单等待 | 不急于成交 |
| **止损单** | 触发价格后转为市价单 | 风险控制 |
| **冰山单** | 只显示部分数量 | 大单隐藏 |
| **FOK** | 全部成交或全部取消 | 确保完整执行 |
| **IOC** | 立即成交或取消 | 避免挂单 |

## 4. 数据获取方式

In [ ]:
# 4.1 导入依赖

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ccxt
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')

print('✅ 依赖库导入成功！')

In [ ]:
# 4.2 REST API 获取订单簿

exchange = ccxt.binance({'enableRateLimit': True})
exchange.load_markets()

def fetch_orderbook(symbol: str = 'BTC/USDT', limit: int = 20) -> dict:
    """
    获取订单簿数据
    """
    orderbook = exchange.fetch_order_book(symbol, limit=limit)
    return {
        'symbol': symbol,
        'timestamp': orderbook['timestamp'],
        'bids': pd.DataFrame(orderbook['bids'], columns=['price', 'amount']),
        'asks': pd.DataFrame(orderbook['asks'], columns=['price', 'amount']),
        'bid': orderbook['bids'][0][0] if orderbook['bids'] else None,
        'ask': orderbook['asks'][0][0] if orderbook['asks'] else None,
    }

# 获取 BTC 订单簿
ob = fetch_orderbook('BTC/USDT', limit=20)

print(f'📊 BTC/USDT 订单簿')
print(f'   时间: {datetime.fromtimestamp(ob["timestamp"]/1000)}')
print(f'   买一: ${ob["bid"]:,.2f}')
print(f'   卖一: ${ob["ask"]:,.2f}')
print(f'   价差: ${ob["ask"] - ob["bid"]:.2f} ({(ob["ask"] - ob["bid"])/ob["bid"]*100:.4f}%)')

In [ ]:
# 4.3 订单簿数据展示

print('\n📊 买单（Bids）:')
print(ob['bids'].head(10).to_string(index=False))

print('\n📊 卖单（Asks）:')
print(ob['asks'].head(10).to_string(index=False))

---

# 第二部分：订单簿分析

---

## 5. 买卖价差分析

In [ ]:
# 5.1 价差计算

class SpreadAnalyzer:
    """
    买卖价差分析器
    """
    def __init__(self, orderbook: dict):
        self.bid = orderbook['bid']
        self.ask = orderbook['ask']
        self.bids = orderbook['bids']
        self.asks = orderbook['asks']
    
    def absolute_spread(self) -> float:
        """绝对价差"""
        return self.ask - self.bid
    
    def relative_spread(self) -> float:
        """相对价差（基点）"""
        mid = (self.ask + self.bid) / 2
        return (self.ask - self.bid) / mid * 10000
    
    def mid_price(self) -> float:
        """中间价"""
        return (self.ask + self.bid) / 2
    
    def weighted_mid_price(self) -> float:
        """加权中间价（考虑深度）"""
        bid_vol = self.bids['amount'].iloc[0]
        ask_vol = self.asks['amount'].iloc[0]
        return (self.bid * ask_vol + self.ask * bid_vol) / (bid_vol + ask_vol)
    
    def effective_spread(self, trade_size: float) -> float:
        """有效价差（考虑吃单深度）"""
        # 计算买入成本
        buy_cost = self._calculate_execution_price(self.asks, trade_size)
        # 计算卖出收入
        sell_price = self._calculate_execution_price(self.bids, trade_size, is_buy=False)
        
        mid = self.mid_price()
        return ((buy_cost - mid) + (mid - sell_price)) / mid * 10000
    
    def _calculate_execution_price(self, levels: pd.DataFrame, size: float, is_buy: bool = True) -> float:
        """计算执行价格"""
        remaining = size
        total_cost = 0
        
        for _, row in levels.iterrows():
            if remaining <= 0:
                break
            filled = min(remaining, row['amount'])
            total_cost += filled * row['price']
            remaining -= filled
        
        return total_cost / size if size > 0 else 0

# 分析价差
analyzer = SpreadAnalyzer(ob)

print('📊 价差分析')
print('=' * 40)
print(f'绝对价差: ${analyzer.absolute_spread():.2f}')
print(f'相对价差: {analyzer.relative_spread():.2f} bps')
print(f'中间价: ${analyzer.mid_price():,.2f}')
print(f'加权中间价: ${analyzer.weighted_mid_price():,.2f}')
print(f'有效价差 (1 BTC): {analyzer.effective_spread(1):.2f} bps')
print(f'有效价差 (10 BTC): {analyzer.effective_spread(10):.2f} bps')

## 6. 深度不平衡指标

In [ ]:
# 6.1 订单簿不平衡

def calculate_imbalance(bids: pd.DataFrame, asks: pd.DataFrame, levels: int = 5) -> dict:
    """
    计算订单簿不平衡指标
    
    正值：买盘强，可能上涨
    负值：卖盘强，可能下跌
    """
    bid_volume = bids['amount'].head(levels).sum()
    ask_volume = asks['amount'].head(levels).sum()
    
    # 简单不平衡
    simple_imbalance = (bid_volume - ask_volume) / (bid_volume + ask_volume)
    
    # 加权不平衡（距离最优价越近权重越大）
    bid_weighted = sum((levels - i) * bids['amount'].iloc[i] for i in range(min(levels, len(bids))))
    ask_weighted = sum((levels - i) * asks['amount'].iloc[i] for i in range(min(levels, len(asks))))
    weighted_imbalance = (bid_weighted - ask_weighted) / (bid_weighted + ask_weighted)
    
    return {
        'bid_volume': bid_volume,
        'ask_volume': ask_volume,
        'simple_imbalance': simple_imbalance,
        'weighted_imbalance': weighted_imbalance,
        'interpretation': '看涨' if simple_imbalance > 0.1 else '看跌' if simple_imbalance < -0.1 else '中性'
    }

imbalance = calculate_imbalance(ob['bids'], ob['asks'])

print('📊 订单簿不平衡分析')
print('=' * 40)
print(f'买盘总量 (前5档): {imbalance["bid_volume"]:.4f}')
print(f'卖盘总量 (前5档): {imbalance["ask_volume"]:.4f}')
print(f'简单不平衡: {imbalance["simple_imbalance"]:.4f}')
print(f'加权不平衡: {imbalance["weighted_imbalance"]:.4f}')
print(f'解读: {imbalance["interpretation"]}')

## 7. 订单流分析

In [ ]:
# 7.1 获取最近成交

def fetch_recent_trades(symbol: str = 'BTC/USDT', limit: int = 100) -> pd.DataFrame:
    """
    获取最近成交记录
    """
    trades = exchange.fetch_trades(symbol, limit=limit)
    
    df = pd.DataFrame([{
        'timestamp': t['timestamp'],
        'datetime': t['datetime'],
        'price': t['price'],
        'amount': t['amount'],
        'side': t['side'],
        'value': t['price'] * t['amount']
    } for t in trades])
    
    return df

df_trades = fetch_recent_trades('BTC/USDT', limit=100)

print(f'📊 最近 {len(df_trades)} 笔成交')
print(df_trades.head(10).to_string(index=False))

In [ ]:
# 7.2 订单流统计

def analyze_order_flow(trades: pd.DataFrame) -> dict:
    """
    分析订单流
    """
    buy_trades = trades[trades['side'] == 'buy']
    sell_trades = trades[trades['side'] == 'sell']
    
    buy_volume = buy_trades['amount'].sum()
    sell_volume = sell_trades['amount'].sum()
    
    buy_value = buy_trades['value'].sum()
    sell_value = sell_trades['value'].sum()
    
    # 计算净流入
    net_flow = buy_value - sell_value
    
    # 计算订单流不平衡
    flow_imbalance = (buy_volume - sell_volume) / (buy_volume + sell_volume)
    
    return {
        'buy_count': len(buy_trades),
        'sell_count': len(sell_trades),
        'buy_volume': buy_volume,
        'sell_volume': sell_volume,
        'buy_value': buy_value,
        'sell_value': sell_value,
        'net_flow': net_flow,
        'flow_imbalance': flow_imbalance
    }

flow_stats = analyze_order_flow(df_trades)

print('📊 订单流分析')
print('=' * 40)
print(f'买单数量: {flow_stats["buy_count"]}')
print(f'卖单数量: {flow_stats["sell_count"]}')
print(f'买入量: {flow_stats["buy_volume"]:.4f} BTC')
print(f'卖出量: {flow_stats["sell_volume"]:.4f} BTC')
print(f'净流入: ${flow_stats["net_flow"]:,.2f}')
print(f'流量不平衡: {flow_stats["flow_imbalance"]:.4f}')

## 8. VWAP/TWAP 计算

In [ ]:
# 8.1 VWAP 计算

def calculate_vwap(trades: pd.DataFrame) -> float:
    """
    计算成交量加权平均价格（VWAP）
    
    VWAP = Σ(Price × Volume) / Σ(Volume)
    """
    return (trades['price'] * trades['amount']).sum() / trades['amount'].sum()

def calculate_twap(trades: pd.DataFrame) -> float:
    """
    计算时间加权平均价格（TWAP）
    
    TWAP = Σ(Price) / N
    """
    return trades['price'].mean()

vwap = calculate_vwap(df_trades)
twap = calculate_twap(df_trades)
current_price = df_trades['price'].iloc[-1]

print('📊 加权平均价格')
print('=' * 40)
print(f'VWAP: ${vwap:,.2f}')
print(f'TWAP: ${twap:,.2f}')
print(f'当前价: ${current_price:,.2f}')
print(f'价格 vs VWAP: {(current_price/vwap - 1)*100:+.4f}%')

---

# 第三部分：微观结构指标

---

## 9. 流动性指标

In [ ]:
# 9.1 流动性指标计算

class LiquidityMetrics:
    """
    流动性指标计算器
    """
    @staticmethod
    def kyle_lambda(trades: pd.DataFrame) -> float:
        """
        Kyle's Lambda: 价格对订单流的敏感度
        
        较高的 Lambda 表示流动性较差
        """
        # 计算价格变化
        price_changes = trades['price'].diff().dropna()
        # 计算签名的成交量
        signed_volume = trades['amount'].copy()
        signed_volume[trades['side'] == 'sell'] *= -1
        signed_volume = signed_volume.iloc[1:]
        
        if len(price_changes) < 2:
            return 0
        
        # 回归计算 Lambda
        cov = np.cov(signed_volume, price_changes)[0, 1]
        var = np.var(signed_volume)
        
        return cov / var if var != 0 else 0
    
    @staticmethod
    def amihud_illiquidity(trades: pd.DataFrame) -> float:
        """
        Amihud 非流动性指标
        
        Amihud = |Return| / Volume
        """
        returns = trades['price'].pct_change().abs().dropna()
        volumes = trades['value'].iloc[1:]
        
        return (returns / volumes).mean() * 1e6  # 放大以便阅读
    
    @staticmethod
    def depth_at_levels(bids: pd.DataFrame, asks: pd.DataFrame, bps: float = 10) -> dict:
        """
        计算指定基点范围内的深度
        """
        mid = (bids['price'].iloc[0] + asks['price'].iloc[0]) / 2
        range_pct = bps / 10000
        
        bid_depth = bids[bids['price'] >= mid * (1 - range_pct)]['amount'].sum()
        ask_depth = asks[asks['price'] <= mid * (1 + range_pct)]['amount'].sum()
        
        return {
            'bid_depth': bid_depth,
            'ask_depth': ask_depth,
            'total_depth': bid_depth + ask_depth
        }

# 计算流动性指标
liq = LiquidityMetrics()

kyle = liq.kyle_lambda(df_trades)
amihud = liq.amihud_illiquidity(df_trades)
depth = liq.depth_at_levels(ob['bids'], ob['asks'], bps=10)

print('📊 流动性指标')
print('=' * 40)
print(f"Kyle's Lambda: {kyle:.6f}")
print(f'Amihud 非流动性: {amihud:.4f}')
print(f'10bps 内买盘深度: {depth["bid_depth"]:.4f} BTC')
print(f'10bps 内卖盘深度: {depth["ask_depth"]:.4f} BTC')

## 10. 市场冲击模型

In [ ]:
# 10.1 市场冲击估算

def estimate_market_impact(orderbook: dict, order_size: float, side: str = 'buy') -> dict:
    """
    估算市场冲击
    
    参数:
        orderbook: 订单簿数据
        order_size: 订单大小
        side: 'buy' 或 'sell'
    """
    levels = orderbook['asks'] if side == 'buy' else orderbook['bids']
    
    remaining = order_size
    total_cost = 0
    filled_levels = []
    
    for _, row in levels.iterrows():
        if remaining <= 0:
            break
        
        filled = min(remaining, row['amount'])
        total_cost += filled * row['price']
        remaining -= filled
        filled_levels.append({
            'price': row['price'],
            'filled': filled
        })
    
    if order_size == 0:
        return {'error': 'Order size is 0'}
    
    avg_price = total_cost / (order_size - remaining)
    best_price = levels['price'].iloc[0]
    
    # 市场冲击 = (平均执行价 - 最优价) / 最优价
    impact = (avg_price - best_price) / best_price * 10000  # bps
    if side == 'sell':
        impact = -impact
    
    return {
        'order_size': order_size,
        'side': side,
        'best_price': best_price,
        'avg_execution_price': avg_price,
        'market_impact_bps': impact,
        'filled_levels': len(filled_levels),
        'unfilled': remaining
    }

# 估算不同订单大小的市场冲击
print('📊 市场冲击估算')
print('=' * 60)
print(f'{"订单大小":<12} {"平均执行价":<15} {"冲击 (bps)":<12} {"吃单档数":<10}')
print('-' * 60)

for size in [0.1, 0.5, 1, 5, 10, 50]:
    impact = estimate_market_impact(ob, size, 'buy')
    if 'error' not in impact:
        print(f"{size:<12} ${impact['avg_execution_price']:<14,.2f} {impact['market_impact_bps']:<12.2f} {impact['filled_levels']:<10}")

## 11. 价格发现机制

### 11.1 价格发现的来源

```
┌─────────────────────────────────────────────────────────────────┐
│                      价格发现机制                               │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  1. 订单流信息                                                 │
│     └── 主动买入/卖出反映市场方向                              │
│                                                                 │
│  2. 订单簿不平衡                                               │
│     └── 买卖盘深度差异预示价格变动                             │
│                                                                 │
│  3. 大单信号                                                   │
│     └── 大额订单可能反映知情交易                               │
│                                                                 │
│  4. 跨市场信息                                                 │
│     └── 期货、期权等衍生品市场的价格信号                       │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

---

# 第四部分：高频策略

---

## 12. 做市策略

In [ ]:
# 12.1 简单做市策略

class SimpleMarketMaker:
    """
    简单做市策略
    
    在买卖两边挂单，赚取价差
    """
    def __init__(self, spread_bps: float = 10, order_size: float = 0.1):
        self.spread_bps = spread_bps
        self.order_size = order_size
        self.position = 0
        self.pnl = 0
    
    def calculate_quotes(self, mid_price: float) -> dict:
        """
        计算报价
        """
        half_spread = mid_price * (self.spread_bps / 20000)
        
        return {
            'bid_price': mid_price - half_spread,
            'ask_price': mid_price + half_spread,
            'bid_size': self.order_size,
            'ask_size': self.order_size
        }
    
    def simulate_fill(self, quotes: dict, market_price: float) -> dict:
        """
        模拟成交
        """
        fills = []
        
        # 检查买单是否成交
        if market_price <= quotes['bid_price']:
            self.position += quotes['bid_size']
            self.pnl -= quotes['bid_price'] * quotes['bid_size']
            fills.append({'side': 'buy', 'price': quotes['bid_price'], 'size': quotes['bid_size']})
        
        # 检查卖单是否成交
        if market_price >= quotes['ask_price']:
            self.position -= quotes['ask_size']
            self.pnl += quotes['ask_price'] * quotes['ask_size']
            fills.append({'side': 'sell', 'price': quotes['ask_price'], 'size': quotes['ask_size']})
        
        return {'fills': fills, 'position': self.position, 'pnl': self.pnl}

# 示例
mm = SimpleMarketMaker(spread_bps=10, order_size=0.1)
mid = analyzer.mid_price()
quotes = mm.calculate_quotes(mid)

print('📊 做市策略报价')
print('=' * 40)
print(f'中间价: ${mid:,.2f}')
print(f'买价: ${quotes["bid_price"]:,.2f}')
print(f'卖价: ${quotes["ask_price"]:,.2f}')
print(f'价差: {(quotes["ask_price"]/quotes["bid_price"]-1)*10000:.1f} bps')

## 13. 统计套利

In [ ]:
# 13.1 配对交易信号

def calculate_zscore(spread: pd.Series, window: int = 20) -> pd.Series:
    """
    计算价差的 Z-Score
    """
    mean = spread.rolling(window).mean()
    std = spread.rolling(window).std()
    return (spread - mean) / std

# 模拟两个相关资产的价差
np.random.seed(42)
n = 100
spread = pd.Series(np.cumsum(np.random.randn(n) * 0.01) + np.random.randn(n) * 0.1)

zscore = calculate_zscore(spread)

print('📊 配对交易信号')
print('=' * 40)
print(f'当前价差: {spread.iloc[-1]:.4f}')
print(f'当前 Z-Score: {zscore.iloc[-1]:.4f}')
print(f'信号: ', end='')
if zscore.iloc[-1] > 2:
    print('做空价差（卖A买B）')
elif zscore.iloc[-1] < -2:
    print('做多价差（买A卖B）')
else:
    print('无信号')

## 14. 动量/均值回归

In [ ]:
# 14.1 订单簿动量信号

def orderbook_momentum(ob_imbalance_history: list, threshold: float = 0.3) -> str:
    """
    基于订单簿不平衡的动量信号
    """
    if len(ob_imbalance_history) < 3:
        return 'HOLD'
    
    recent_imbalance = np.mean(ob_imbalance_history[-3:])
    
    if recent_imbalance > threshold:
        return 'BUY'  # 买盘持续强势
    elif recent_imbalance < -threshold:
        return 'SELL'  # 卖盘持续强势
    else:
        return 'HOLD'

# 模拟订单簿不平衡历史
imbalance_history = [0.1, 0.15, 0.2, 0.25, 0.3]

signal = orderbook_momentum(imbalance_history)
print(f'📊 订单簿动量信号: {signal}')

---

# 第五部分：实战系统

---

## 15. WebSocket 实时数据

In [ ]:
# 15.1 WebSocket 订单簿处理示例（伪代码）

print('''
📊 WebSocket 订单簿连接示例

import websocket
import json

def on_message(ws, message):
    data = json.loads(message)
    
    if 'b' in data:  # bids
        bids = data['b']
    if 'a' in data:  # asks
        asks = data['a']
    
    # 更新本地订单簿
    update_local_orderbook(bids, asks)
    
    # 计算指标
    imbalance = calculate_imbalance(local_bids, local_asks)
    
    # 生成信号
    if imbalance > 0.3:
        generate_signal('BUY')

# 连接 Binance WebSocket
ws = websocket.WebSocketApp(
    "wss://stream.binance.com:9443/ws/btcusdt@depth@100ms",
    on_message=on_message
)
ws.run_forever()
''')

print('\n💡 提示: WebSocket 代码需要在异步环境中运行')

## 16. 订单簿可视化

In [ ]:
# 16.1 深度图

def plot_depth_chart(orderbook: dict):
    """
    绘制订单簿深度图
    """
    bids = orderbook['bids'].copy()
    asks = orderbook['asks'].copy()
    
    # 计算累计量
    bids['cumulative'] = bids['amount'].cumsum()
    asks['cumulative'] = asks['amount'].cumsum()
    
    fig = go.Figure()
    
    # 买单
    fig.add_trace(go.Scatter(
        x=bids['price'],
        y=bids['cumulative'],
        fill='tozeroy',
        name='买单',
        line=dict(color='green'),
        fillcolor='rgba(0, 255, 0, 0.3)'
    ))
    
    # 卖单
    fig.add_trace(go.Scatter(
        x=asks['price'],
        y=asks['cumulative'],
        fill='tozeroy',
        name='卖单',
        line=dict(color='red'),
        fillcolor='rgba(255, 0, 0, 0.3)'
    ))
    
    fig.update_layout(
        title='订单簿深度图',
        xaxis_title='价格',
        yaxis_title='累计数量',
        height=400
    )
    
    return fig

fig = plot_depth_chart(ob)
fig.show()

In [ ]:
# 16.2 订单簿热力图

def plot_orderbook_heatmap(orderbook: dict):
    """
    绘制订单簿热力图
    """
    bids = orderbook['bids'].head(20)
    asks = orderbook['asks'].head(20)
    
    all_prices = pd.concat([bids['price'], asks['price']])
    all_amounts = pd.concat([
        -bids['amount'],  # 买单用负数表示
        asks['amount']
    ])
    
    colors = ['green' if a < 0 else 'red' for a in all_amounts]
    
    fig = go.Figure(go.Bar(
        x=all_prices,
        y=all_amounts.abs(),
        marker_color=colors,
        text=[f'{abs(a):.4f}' for a in all_amounts],
        textposition='outside'
    ))
    
    fig.update_layout(
        title='订单簿柱状图',
        xaxis_title='价格',
        yaxis_title='数量',
        height=400
    )
    
    return fig

fig = plot_orderbook_heatmap(ob)
fig.show()

## 17. 执行算法

In [ ]:
# 17.1 TWAP 执行算法

class TWAPExecutor:
    """
    TWAP (Time-Weighted Average Price) 执行算法
    
    将大订单分割成小订单，在指定时间内均匀执行
    """
    def __init__(self, total_size: float, duration_minutes: int, num_slices: int = 10):
        self.total_size = total_size
        self.duration_minutes = duration_minutes
        self.num_slices = num_slices
        self.slice_size = total_size / num_slices
        self.interval_seconds = (duration_minutes * 60) / num_slices
    
    def get_schedule(self) -> list:
        """
        获取执行计划
        """
        schedule = []
        for i in range(self.num_slices):
            schedule.append({
                'slice': i + 1,
                'time_offset_seconds': i * self.interval_seconds,
                'size': self.slice_size
            })
        return schedule

# 示例
twap = TWAPExecutor(total_size=10, duration_minutes=30, num_slices=10)
schedule = twap.get_schedule()

print('📊 TWAP 执行计划')
print('=' * 50)
print(f'总量: {twap.total_size} BTC')
print(f'执行时间: {twap.duration_minutes} 分钟')
print(f'分片数: {twap.num_slices}')
print(f'每片大小: {twap.slice_size} BTC')
print(f'执行间隔: {twap.interval_seconds} 秒')
print('\n执行计划:')
for s in schedule[:5]:
    print(f"  第{s['slice']}片: {s['time_offset_seconds']:.0f}秒后执行 {s['size']} BTC")

In [ ]:
# 17.2 VWAP 执行算法

class VWAPExecutor:
    """
    VWAP (Volume-Weighted Average Price) 执行算法
    
    根据历史成交量分布来安排订单执行
    """
    def __init__(self, total_size: float, volume_profile: list):
        self.total_size = total_size
        self.volume_profile = volume_profile
        self.total_volume = sum(volume_profile)
    
    def get_schedule(self) -> list:
        """
        根据成交量分布生成执行计划
        """
        schedule = []
        for i, vol in enumerate(self.volume_profile):
            weight = vol / self.total_volume
            size = self.total_size * weight
            schedule.append({
                'period': i + 1,
                'volume_weight': weight,
                'size': size
            })
        return schedule

# 示例：假设的小时成交量分布
volume_profile = [100, 80, 60, 50, 70, 90, 120, 150]  # 8个时段的成交量
vwap = VWAPExecutor(total_size=10, volume_profile=volume_profile)
schedule = vwap.get_schedule()

print('📊 VWAP 执行计划')
print('=' * 50)
for s in schedule:
    print(f"  时段{s['period']}: 权重 {s['volume_weight']*100:.1f}%, 执行 {s['size']:.4f} BTC")

---

## 总结

### 核心要点

1. **订单簿** 是理解市场微观结构的基础

2. **关键指标**：
   - 买卖价差
   - 深度不平衡
   - 订单流
   - 市场冲击

3. **高频策略**：
   - 做市策略
   - 统计套利
   - 动量策略

4. **执行算法**：
   - TWAP：时间均匀执行
   - VWAP：按成交量分布执行

### 实战建议

- 使用 WebSocket 获取实时数据
- 关注流动性变化
- 控制市场冲击
- 完善风险管理

### 下一步学习

- [ ] 期权与永续合约组合策略